### Pre-processing the The Stack dataset for fill-in-the-middle


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from datasets import load_dataset

full_dataset = load_dataset("bigcode/the-stack", data_dir="data/c-sharp", split="train", streaming=True, use_auth_token=True)

Using custom data configuration bigcode--the-stack-738b5ba2c954e262


In [14]:
%pip install tree_sitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.6/487.6 kB 30.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
!git clone https://github.com/tree-sitter/tree-sitter-c-sharp.git vendor/tree-sitter-c-sharp

Cloning into 'vendor/tree-sitter-c-sharp'...
remote: Enumerating objects: 3471, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 3471 (delta 157), reused 223 (delta 147), pack-reused 3228
Receiving objects: 100% (3471/3471), 104.19 MiB | 24.92 MiB/s, done.
Resolving deltas: 100% (2545/2545), done.


In [38]:
from tree_sitter import Language, Parser

Language.build_library(
    # Store the library in the `build` directory
    "./build/my-languages.so",
    # Include one or more languages
    ["vendor"],
)

False

In [39]:
C_SHARP_LANGUAGE = Language("./build/my-languages.so", "c_sharp")
parser = Parser()
parser.set_language(C_SHARP_LANGUAGE)

In [40]:
tree = parser.parse(
    bytes(
        """
public static void main(string[] args) {}
""",
        "utf8",
    )
)

In [52]:
def make_move(cursor, came_up, fn):
    if not came_up:
        fn(cursor)
        if (cursor.goto_first_child()):
            make_move(cursor, False, fn)
        elif (cursor.goto_next_sibling()):
            make_move(cursor, False, fn)
        elif (cursor.goto_parent()):
            make_move(cursor, True, fn)
    else:
        if (cursor.goto_next_sibling()):
            make_move(cursor, False, fn)
        elif (cursor.goto_parent()):
            make_move(cursor, True, fn)

cursor = tree.walk()

cursor.node
#make_move(cursor, False, lambda c: print(c.node.type))


<Node type=compilation_unit, start_point=(1, 0), end_point=(2, 0)>

In [4]:
%pip install sympy
from sympy import ln

print(tok.vocab_size)
print(loss)
print(-ln(1 / tok.vocab_size))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 56.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 61.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
50265
Seq2SeqLMOutput(loss=tensor(11.1444, grad_fn=<NllLossBackward0>), logits=tensor([[[-0.3565,  0.0000,  0.6234,  ...,  0.1350, -1.9323, -0.7257],
         [-0.1440,  0.0000,  0.1526,  ...,  0.6898, -0.9209, -0.6615],
         [-1.0764,  0.0000, -0.6017,  ...,  0.8245, -1.0480, -0.2220],
         ...,
         [-1.2544,  0.0000, -0.7136,  ...,  1.2973, -0.2436,  0.0962],
         [-0.8492,  0.0000,  0.2782,  ...,  0.8116, -0.8380, -0.0363],
         [-1.0910,  0.0000, -0.2373,  ...,  0.2362, -1.2395,  0.6950]],

        [[-0.8653,  0.0000,  0.8984,  ...,  0.8659, -0.6341, -0.8638],
         [-0.2101,  0.0000, -0.4880,  ...,  0.0982, -0.9561,  0.0177],
         [-0.3180,  0.0000, -1.3024,  ..., -0.0603, -0.3040,  0.1073],
         ...,
         [-0.580

In [55]:
trainer = Trainer(model=mod, tokenizer=tok, train_dataset=)
trainer.train()

ValueError: Trainer: training requires a train_dataset.

We will need to do a few things, including:

- Adding a series of special tokens for fill-in-the-middle that can be read by the tokenizer
- Move around text from the training dataset following a principle of the paper by [Bavarian et. al](https://arxiv.org/abs/2207.14255)
- Pre-process the training and evaluation datasets with the fill-in-the-middle tokens


In [1]:
%pip install datasets transformers numpy

Note: you may need to restart the kernel to use updated packages.


First, let's create the new special tokens.


In [48]:
from transformers import BartTokenizerFast

additional_special_tokens = ["<PRE>", "<SUF>", "<MID>"]
model_name = "facebook/bart-large"

tokenizer = BartTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
tokenizer.add_tokens(additional_special_tokens, special_tokens=True)

Next, we should randomly chop up our testing data, and append our tokens.


In [3]:
from datasets import load_dataset

full_dataset = load_dataset("microsoft/LCC_csharp")

c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


We'll reduce the length of the datasets so that it's easier to see that our process is working


In [20]:
small_training_dataset = full_dataset["train"].shuffle(42).select(range(1))
small_eval_dataset = full_dataset["validation"].shuffle(42).select(range(1))

Let's define a function that will separate our input data into pre, suf and mid sections, but that doesn't split words and doesn't go past the last word/token.
Some code is taken directly from the research paper referenced earlier.


In [21]:
import random


def randomly_split(tokens: list[int]) -> tuple[list[int], list[int], list[int]]:
    total_size = len(tokens)

    size1 = random.randint(1, total_size - 2)
    size2 = random.randint(1, total_size - size1 - 1)

    sublist1 = tokens[1:size1]
    sublist2 = tokens[size1 : size1 + size2]
    sublist3 = tokens[size1 + size2 :]

    return sublist1, sublist2, sublist3


def token_level_psm_fim(tokens: list[int]) -> list[int]:
    prefix, middle, suffix = randomly_split(tokens)
    return [
        tokenizer.bos_token_id,
        *tokenizer("<PRE>", add_special_tokens=False)["input_ids"],
        *prefix,
        *tokenizer("<SUF>", add_special_tokens=False)["input_ids"],
        *suffix,
        *tokenizer("<MID>", add_special_tokens=False)["input_ids"],
        *middle,
        tokenizer.eos_token_id,
    ]


small_training_dataset = small_training_dataset.map(
    lambda x: tokenizer(x["context"], add_special_tokens=False),
    remove_columns=full_dataset["train"].column_names,
    num_proc=4,
)
small_training_dataset = small_training_dataset.map(
    lambda x: {"input_ids": token_level_psm_fim(x["input_ids"])}, num_proc=4
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [23]:
small_training_dataset = small_training_dataset.map(
    lambda x: tokenizer(
        tokenizer.decode(x["input_ids"]),
        add_special_tokens=False,
        padding=True,
        max_length=13000,
    )
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Now we should build out our validation dataset. This will be the context row appended with the gt row, with our special tokens added in the correct locations.


In [24]:
def build_training_data(examples):
    to_tokenized = "<PRE>" + examples["context"] + "<SUF><MID>" + examples["gt"]
    tokenized = tokenizer(to_tokenized, padding=True, max_length=13000)
    return tokenized


small_eval_dataset = small_eval_dataset.map(
    build_training_data, remove_columns=full_dataset["train"].column_names, num_proc=4
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [25]:
train_labels = small_training_dataset["input_ids"]
small_training_dataset = small_training_dataset.add_column(
    name="labels", column=train_labels
)

c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [26]:
eval_labels = small_eval_dataset["input_ids"]
small_eval_dataset = small_eval_dataset.add_column(name="labels", column=eval_labels)

In [27]:
print(len(small_training_dataset[0]["input_ids"]))
print(len(small_training_dataset[0]["attention_mask"]))
print(len(small_training_dataset[0]["labels"]))
print(len(small_eval_dataset[0]["input_ids"]))
print(len(small_eval_dataset[0]["attention_mask"]))
print(len(small_eval_dataset[0]["labels"]))

12840
12840
12840
3446
3446
3446


If we've done everything right, our training dataset token length and our validation dataset token length should be the same.


Now that we have a method by which we can split data along lines made by tokens, we can start feeding this into our trainer.


In [32]:
from transformers import (
    TrainingArguments,
    Trainer,
    BartForConditionalGeneration,
    BartConfig,
    DataCollatorWithPadding,
)

config = BartConfig()
mod = BartForConditionalGeneration(config)
collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

# define the trainer and start training
trainer = Trainer(
    model=mod,
    args=training_args,
    train_dataset=small_training_dataset,
    # eval_dataset=small_eval_dataset,
    # data_collator=collator,
)

In [47]:
small_training_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [46]:
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

IndexError: index out of range in self